In [1]:
import pandas as pd
import numpy as np
import gc
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
# Reading in data
prior_df = pd.read_csv('order_products__prior.csv', dtype={'order_id': np.uint32,
           'product_id': np.uint16, 'reordered': np.uint8, 'add_to_cart_order': np.uint8})

train_df = pd.read_csv('order_products__train.csv', dtype={'order_id': np.uint32,
           'product_id': np.uint16, 'reordered': np.int8, 'add_to_cart_order': np.uint8 })

orders_df = pd.read_csv('orders.csv', dtype={'order_hour_of_day': np.uint8,
           'order_number': np.uint8, 'order_id': np.uint32, 'user_id': np.uint32,
           'order_dow': np.uint8, 'days_since_prior_order': np.float16})

products_df = pd.read_csv('products.csv', dtype={'product_id': np.uint16,
            'aisle_id': np.uint8, 'department_id': np.uint8},
             usecols=['product_id', 'aisle_id', 'department_id'])

In [3]:
# Combining prior and train data to do feature engineering. They will be separated later for train and test splitting
prior_train_df=pd.concat([prior_df,train_df])

In [4]:
# Checking the number of orders in each set. 
orders_df["eval_set"].value_counts()

prior    3214874
train     131209
test       75000
Name: eval_set, dtype: int64

In [6]:
# Checking the number of unique users in each set. Note that prior has all the users. Some of these users are present in the train set
# While the rest are present in the test set. 
orders_df.groupby("eval_set")["user_id"].nunique()

eval_set
prior    206209
test      75000
train    131209
Name: user_id, dtype: int64

Currently our dataframe contains data in the below format

| User  | Order_number   | Product   | Reordered   |
|---|---|---|---|
| 1  | 1  |  A | 0  |
| 1  |  1 |  B |  0 | 
| 1  |  1 |  C |  0 |
| 1  | 2  |  B | 1  |
| 1  | 2  |  D | 0  |


But because our objective is to predict which of the products a customer has bought in the past will be reordered in his latest order, it will be better to have all the products which a customer has bought in the past aggregated and predict 0 or 1 against each product . For instance let us suppose data in the following format 

| User  | Order_number   | Product   | Ordered(1)/Not Ordered (0)|
|---|---|---|---|
| 1  | 1  |  A | 1  |
| 1  |  1 |  B | 1 | 
| 1  |  1 |  C |  1 |
| 1  |  1 |  D |  0 |
| 1  | 2  |  A | 0  |
| 1  | 2  |  B | 1  |
| 1  | 2  |  C | 0  |
| 1  | 2  |  D | 1  |



The aggregate of all the products customer has ordered in his history is {A,B,C,D}

Lets say the above customer's latest order is order_number = 3 then our objective will be to predict ordered or not-ordered for the products A,B,C,D

| User  | Order_number   | Product   | Ordered(1)/Not Ordered (0)|
|---|---|---|---|
| 1  | 1  |  A | 1  |
| 1  |  1 |  B | 1 | 
| 1  |  1 |  C |  1 |
| 1  |  1 |  D |  0 |
| 1  | 2  |  A | 0  |
| 1  | 2  |  B | 1  |
| 1  | 2  |  C | 0  |
| 1  | 2  |  D | 1  |
| 1  | 3  |  A | ?  |
| 1  | 3  |  B | ?  |
| 1  | 3  |  C | ?  |
| 1  | 3  |  D | ?  |

So instead of focusing on the reordered variable, for a particular customer's latest order we will try to predict either 0 or 1 against each product which the customer has ever ordered in the past. Now let us build the above user-product table as outlined above

In [ ]:
orders_prior_train_df=prior_train_df.merge(orders_df,on="order_id")
up_user_products=orders_prior_train_df[["user_id","product_id"]].drop_duplicates()

#Aggregate all the products which a customer has ordered into a set
temp=orders_prior_train_df.groupby(["user_id","order_id"],as_index=False).agg({"product_id":(lambda x:set(x))})
temp=temp.merge(up_user_products,on="user_id").rename(columns={"product_id_x":"order_id_products","product_id_y":"product_id"})

#Create table with either 0 or 1 against each user, product id (product id from the aggregated set) as outlined in discussion above
temp_df=temp.head(1)
temp_df["in_cart"]=2
temp_df["user_id"]=0
temp_df=temp_df.drop("order_id_products",axis=1)

#Please note the following steps takes around 40GB of RAM
chunksize=int(temp.shape[0]/20)
i=0
for start in range(0,temp.shape[0],chunksize):
    i=i+1
    print(i)
    df_subset=temp.iloc[start:start+chunksize]
    df_subset["in_cart"]=(df_subset.apply(lambda row: row['product_id'] in row['order_id_products'], axis=1).astype(int))
    df_subset=df_subset.drop("order_id_products",axis=1)
    temp_df=pd.concat([temp_df,df_subset])
    
temp=orders_prior_train_df[["order_id","u_order_number"]].drop_duplicates()
temp_df=temp_df.merge(temp,on="order_id")

#Pickle this dataset for subsequent use
temp_df.to_pickle("temp_df.pkl")

## Feature Engineering

For our problem, it is natural to think about three types of features: 1) <b> User-related features </b> : Features which are specific to the user in question 2) <b> Product-related features </b>: Features which are specific to a particular product 3) <b> User x Product related features </b>: Features which capture or quantify trends related to a particular user-product combination

<b> Let us start by building user-related features </b>

In [ ]:
# Counting total number of products ("total_user_purchases") ordered by a customer in all his orders (total number, not unique items)
temp=orders_prior_train_df.groupby("user_id")["add_to_cart_order"].count().reset_index().rename(columns={"add_to_cart_order":"total_user_purchases"})

# Adding the "total_user_purchases" feature to orders_prior_train_df
orders_prior_train_df=orders_prior_train_df.merge(temp,on="user_id")

# Calculating the total number of orders ("number_of_orders") for each customer
temp = orders_prior_train_df.groupby("user_id")["order_number"].nunique().reset_index().rename(columns={"order_number":"number_of_orders"})

# Adding the "number_of_orders" feature to orders_prior_train_df
orders_prior_train_df=orders_prior_train_df.merge(temp,on="user_id")

#Calculating the average cart size ("avg_cart_size") of each customer
orders_prior_train_df["avg_cart_size"]=orders_prior_train_df["total_user_purchases"]/orders_prior_train_df["number_of_orders"]

#Calculate average number of days between orders for a customer to get the frequency of ordering 
temp=orders_df.groupby("user_id")["days_since_prior_order"].mean().reset_index().rename(columns={"days_since_prior_order":"average_days_since_prior_order"})

# Adding the "average_days_since_prior_order" feature to orders_prior_train_df
orders_prior_train_df=orders_prior_train_df.merge(temp,on="user_id")

# Calculate how long a user has been a customer ("customer_time") #CHECK THIS
temp=orders_df.groupby("user_id")["days_since_prior_order"].sum().reset_index().rename(columns={"days_since_prior_order":"customer_time"})

# Adding the "customer_time" feature to orders_prior_train_df
orders_prior_train_df=orders_prior_train_df.merge(temp,on="user_id")

#Calculating the average day of the week ("average_dow") on which a customer places an order
temp=orders_df.groupby("user_id")["order_dow"].mean().reset_index().rename(columns={"order_dow":"average_dow"})

# Adding the "average_dow" feature to orders_prior_train_df 
orders_prior_train_df=orders_prior_train_df.merge(temp,on="user_id")

#Calculating the average hour of the day ("average_order_hour_of_day") on which a customer places an order
temp=orders_df.groupby("user_id")["order_hour_of_day"].mean().reset_index().rename(columns={"order_hour_of_day":"average_order_hour_of_day"})

# Adding the "average_order_hour_of_day" feature to orders_prior_train_df 
orders_prior_train_df=orders_prior_train_df.merge(temp,on="user_id")

<b> Now build product-related features </b>

In [ ]:
#Calculating the total times a product is ordered ("total_product_orders")
temp=orders_prior_train_df["product_id"].value_counts().reset_index().rename(columns={"index":"product_id","product_id":"total_product_orders"})

# Adding the "total_product_orders" feature to orders_prior_train_df 
orders_prior_train_df=orders_prior_train_df.merge(temp,on="product_id")

# calculate the average number of days before which users order a particular product ("average_dspo_product") #CHECK THIS
temp=orders_prior_train_df.groupby("product_id")["days_since_prior_order"].mean().reset_index().rename(columns={"days_since_prior_order":"average_dspo_product"})

# Adding the "average_dspo_product" feature to orders_prior_train_df
orders_prior_train_df=orders_prior_train_df.merge(temp,on="product_id")

# Renaming columns to reflect user and product related features
# Adding u, p or up prefix to features which tell us if they are user, product or user x product type features
orders_prior_train_df=orders_prior_train_df.rename(columns={"add_to_cart_order":"up_add_to_cart_order","order_number":"u_order_number","order_dow":"u_order_dow", "order_hour_of_day":"u_order_hour_of_day","days_since_prior_order":"u_days_since_prior_order","total_user_purchases":"u_total_purchases","number_of_orders":"u_number_of_orders","avg_cart_size":"u_avg_cart_size","average_days_since_prior_order":"u_average_dspo","customer_time":"u_customer_time","average_dow":"u_average_dow","average_hour_of_day":"u_average_hod","total_product_orders":"p_total_orders","average_dspo_product":"p_average_dspo"})
orders_prior_train_df=orders_prior_train_df.rename(columns={"average_order_hour_of_day":"u_average_hod"})

#Calculating the average add to cart order ("p_add_to_cart_order")
temp=orders_prior_train_df.groupby("product_id")["up_add_to_cart_order"].mean().reset_index().rename(columns={"up_add_to_cart_order":"p_average_add_to_cart_order"})

# Adding the "p_add_to_cart_order" feature to orders_prior_train_df
orders_prior_train_df=orders_prior_train_df.merge(temp,on="product_id")

#Calculate the number of unique users ordering each product
temp=orders_prior_train_df.groupby("product_id")["user_id"].nunique().to_frame("p_unique_users")

# Adding the "p_unique_users" feature to orders_prior_train_df
orders_prior_train_df=orders_prior_train_df.merge(temp,on="product_id")

In [ ]:
# Pickle the orders_prior_train_df dataset for later use
orders_prior_train_df.to_pickle("orders_prior_train_df.pkl")

## We will now use this dataset for building user x product features

The first set of user x product features which we will build are the so called "streak_weight", "streak" and "streak_mod" features. 

The streak_weight is a feature which captures the recency of the user-product combination. 
Let's say a user has 10 orders in total. The user has ordered a particular product in his 10th order, 9th order and 7th order say then
    - the streak_weight for the 10th order is (1/2)^(number of orders-order number)=(1/2)^(10-10)=1 (Most recent order)
    - the streak_weight for the 9th order is (1/2)^(number of orders-order number) = (1/2)^(10-9)=1/2=0.5
    - the streak_weight for the 7th order is (1/2)^(number of orders-order number) = (1/2)^(10-7)=0.125
    
 The "streak" feature sums over all the streak-weights for a particular user-product combination. THere is one unique "streak" value for a user-product combination. 
 
 The "streak_mod" feature takes the "streak" feature and multiplies it with the probability that that particular item is present in the cart which is the "in_cart_mean" feature. 

In [ ]:
temp_df=pd.read_pickle("temp_df.pkl")
temp_df=temp_df[1:]
temp_df["user_id"]=temp_df["user_id"].astype("uint32")

orders_df=pd.read_csv("orders.csv")

orders_df_train=orders_df[orders_df["eval_set"]=="train"]

temp_df_train=temp_df.merge(orders_df_train,on="order_id",how="inner")
temp_df_train=temp_df_train.rename(columns={"user_id_x":"user_id"}).drop(["user_id_y","order_number"],axis=1)
temp_df_train.to_pickle("temp_df_train_0711.pkl")

temp=orders_prior_df[["user_id","u_number_of_orders"]].drop_duplicates()
temp_df_train=temp_df.merge(temp,on="user_id")
temp=temp_df_train.groupby("user_id")["u_order_number"].max().reset_index()
temp=temp.rename(columns={"u_order_number":"max_order_number_pt"})
temp_df_train=temp_df_train.merge(temp,on="user_id")
temp_df_train=temp_df_train[temp_df_train["u_order_number"]<=temp_df_train["u_number_of_orders"]]
temp_df_train["streak_weight"]=temp_df_train["in_cart"]*((1/2)**(temp_df_train["u_number_of_orders"]-temp_df["u_order_number"]))
temp=temp_df_train.groupby(["user_id","product_id"])["streak_weight"].sum().reset_index().rename(columns={"streak_weight":"streak"})
temp_df_train=temp_df_train.merge(temp,on=["user_id","product_id"])
temp_df_train=temp_df_train[temp_df["streak"]>0]
temp=temp_df_train.groupby(["user_id","product_id"])["in_cart"].mean().reset_index().rename(columns={"in_cart":"in_cart_mean"})
temp_df_train=temp_df_train.merge(temp,on=["user_id","product_id"])
temp_df_train=temp_df_train[temp_df_train["u_order_number"]==temp_df_train["u_number_of_orders"]]
temp_df_train["streak_mod"]=temp_df_train["streak"]*temp_df_train["in_cart_mean"]

temp=orders_prior_df[["order_id","u_order_dow","u_order_hour_of_day","u_days_since_prior_order"]].drop_duplicates()
temp_df_train=temp_df_train.merge(temp,on=["order_id"])

temp=orders_prior_df[["user_id","u_total_purchases",'u_avg_cart_size', 'u_average_dspo',
       'u_customer_time', 'u_average_dow', 'average_order_hour_of_day']].drop_duplicates()
temp_df_train=temp_df_train.merge(temp,on="user_id")
temp_df_train=temp_df_train.rename(columns={"average_order_hour_of_day":"u_average_hod"})
temp_df_train=temp_df_train.drop("streak_weight",axis=1)
temp_df_train=temp_df_train.rename(columns={"u_order_number":"up_order_number","streak":"up_streak","in_cart_mean":"up_in_cart_mean","streak_mod":"up_streak_mod","u_order_dow":"up_order_dow","u_order_hour_of_day":"up_order_hour_of_day","u_days_since_prior_order":"up_days_since_prior_order"})

In [ ]:
# Merging the dataset with user and product features
temp=orders_prior_df[["order_id","u_order_dow","u_order_hour_of_day","u_days_since_prior_order"]].drop_duplicates()
temp_df_train=temp_df.merge(temp,on=["order_id"])
temp=orders_prior_df[["user_id","u_total_purchases",'u_avg_cart_size', 'u_average_dspo',
       'u_customer_time', 'u_average_dow', 'average_order_hour_of_day']].drop_duplicates()
temp_df_train=temp_df_train.merge(temp,on="user_id")
temp_df_train=temp_df_train.rename(columns={"average_order_hour_of_day":"u_average_hod"})
temp_df_train=temp_df_train.drop("streak_weight",axis=1)
temp_df_train=temp_df_train.rename(columns={"u_order_number":"up_order_number","streak":"up_streak","in_cart_mean":"up_in_cart_mean","streak_mod":"up_streak_mod","u_order_dow":"up_order_dow","u_order_hour_of_day":"up_order_hour_of_day","u_days_since_prior_order":"up_days_since_prior_order"})
prod_features=orders_prior_df[["product_id","p_total_orders","p_average_add_to_cart_order","p_unique_users"]].drop_duplicates()
temp_df_train=temp_df_train.merge(prod_features,on="product_id")

In [ ]:
temp_df_train=temp_df.drop(["order_id","in_cart","up_order_number","max_order_number_pt","up_order_dow","up_order_hour_of_day","up_days_since_prior_order"],axis=1)
temp_df_train=temp_df_train.merge(orders_df_train,on="user_id")

orders_df_test=orders_df[orders_df["eval_set"]=="test"]
temp_df_test=temp_df_train.merge(orders_df_test,on="user_id")

In [ ]:
# We have our train and test datasets and we are going to pickle them so that we can use them in future without having to run all the pre-processing steps 
temp_df_train.to_pickle("temp_df_train.pkl")
temp_df_test.to_pickle("temp_df_test.pkl")

In [ ]:
# Read in dataframe with all prior and train orders, product ids, in_cart 
prior_train_df=pd.read_pickle("temp_df.pkl")
prior_train_df=prior_train_df[1:]
orders_df_prior=orders_df[orders_df["eval_set"]=="prior"]
prior_df= prior_train_df.merge(orders_df_prior[["order_id","days_since_prior_order"]],on="order_id")
temp=prior_df.groupby(["user_id","product_id"])["in_cart"].sum().reset_index()
temp=temp[temp["in_cart"]!=0]
temp.rename(columns={"in_cart":"in_cart_sum"},inplace=True)
prior_df=prior_df.merge(temp,on=["user_id","product_id"],how="inner")
prior_df.sort_values(["user_id","product_id","u_order_number"],inplace=True)
prior_df=prior_df.drop("index",axis=1)
cumsums=prior_df.groupby(["user_id","product_id","u_order_number"]).sum().groupby(level=[0,1]).cumsum().reset_index()
cumsums.rename(columns={"days_since_prior_order":"cum_dspo"},inplace=True)
prior_df["cum_dspo"]=cumsums["cum_dspo"]
prior_df['order_time_last'] = prior_df[prior_df["in_cart"]==1].groupby(["user_id","product_id"])["cum_dspo"].transform(np.max) 
prior_df_order_time=prior_df.dropna()
prior_df_order_time=prior_df_order_time[["user_id","product_id","order_time_last"]].drop_duplicates()
prior_df_order_time=prior_df_order_time.reset_index()


In [ ]:
temp=prior_df.groupby(["user_id","product_id"])["in_cart"].sum().reset_index()
prior_df_order_time["in_cart_sum"]=temp["in_cart"]
prior_df_order_time.drop("index",axis=1,inplace=True)
temp_df_train=temp_df_train.merge(prior_df_order_time,on=["user_id","product_id"],how="inner")
temp_df_test=temp_df_test.merge(prior_df_order_time,on=["user_id","product_id"],how="inner")
temp=prior_df.groupby("user_id")["cum_dspo"].max().reset_index()
temp.rename(columns={"cum_dspo":"u_customer_time"},inplace=True)
prior_df=prior_df.drop("u_customer_time",axis=1)

temp_df_train.drop("u_customer_time",axis=1,inplace=True)
temp_df_test.drop("u_customer_time",axis=1,inplace=True)

temp_df_train=temp_df_train.merge(temp,on="user_id")
temp_df_test=temp_df_test.merge(temp,on="user_id")

In [ ]:
temp_df_train=temp_df_train.merge(prior_df_order_time,on=["user_id","product_id"],how="inner")
temp_df_test=temp_df_test.merge(prior_df_order_time,on=["user_id","product_id"],how="inner")

temp_df_train.rename(columns={"order_time_last_x":"order_time_last","in_cart_sum_x":"in_cart_sum"},inplace=True)
temp_df_test.rename(columns={"order_time_last_x":"order_time_last","in_cart_sum_x":"in_cart_sum"},inplace=True)

temp_df_train["up_freq"]=temp_df_train["u_customer_time"]/temp_df_train["in_cart_sum"]
temp_df_test["up_freq"]=temp_df_test["u_customer_time"]/temp_df_test["in_cart_sum"]

temp_df_train["up_days_since"]=temp_df_train["u_customer_time"]-temp_df_train["order_time_last"]+temp_df_train["days_since_prior_order"]
temp_df_test["up_days_since"]=temp_df_test["u_customer_time"]-temp_df_test["order_time_last"]+temp_df_test["days_since_prior_order"]

temp_df_train["up_buildup"]=temp_df_train["up_days_since"]/temp_df_train["up_freq"]
temp_df_test["up_buildup"]=temp_df_test["up_days_since"]/temp_df_test["up_freq"]

temp_df_train[temp_df_train["up_buildup"]==np.nan]
temp_df_train[temp_df_train.isnull().any(axis=1)]

temp_df_train=temp_df_train.fillna(1)
temp_df_test=temp_df_test.fillna(1)
temp_df_train[temp_df_train.isnull().any(axis=1)]

In [ ]:
temp_df_train.to_pickle("temp_df_train.pkl")
temp_df_test.to_pickle("temp_df_test.pkl")

In [ ]:
Xl_df_train=temp_df_train.drop(["user_id","order_id","product_id","in_cart"],axis=1)
yl_df_train=temp_df_train["in_cart"]

## XGBOOST Classifier

In [ ]:
# Fitting a xg boost classifier 
xg_cl_0719 = xgb.XGBClassifier(objective='binary:logistic', n_estimators=50, max_depth=8, seed=123,scale_pos_weight=4 ,colsample_bytree=0.5)
xg_cl_0719.fit(Xl_df_train, yl_df_train)

In [ ]:
Xl_df_test=temp_df_test.drop(["user_id","order_id","product_id"],axis=1)
# Prediction on test data
prediction = xg_cl_0719.predict(Xl_df_test)

#Creating .csv file in format which kaggle expects
orders_df=pd.read_csv("orders.csv")
test_orders_df=orders_df[orders_df["eval_set"]=="test"]

temp_df_test["in_cart"]=prediction
test_set=temp_df_test[temp_df_test["in_cart"]==1]

def mylist(df):
    ramu=df["product_id"]
    if ramu==[]: return 'None'
    return ' '.join(str(i) for i in ramu )

test_set_list=test_set.groupby("order_id").agg({"product_id":(lambda x:list(x))})
test_set_list=test_set_list.reset_index()

test_set_list['products_list']=test_set_list.apply(mylist,axis=1)

kaggle_submission_0719=test_set_list.merge(test_orders_df,on="order_id",how="outer")
kaggle_submission_0719=kaggle_submission_0719.fillna('None')
kaggle_submission_0719=kaggle_submission_0719[["order_id","products_list"]]
kaggle_submission_0719.to_csv("final_submission_xgboostcomp_0719.csv")

#We now have the prediction on the test set 

# Light GBM Classifier

In [1]:
train = lgb.Dataset(data=Xl_df_train, label=yl_df_train)
params = {
    'task': 'train',
    'boosting': 'gbdt',
    'application':'binary',
    'learning_rate':0.1,
    'objective': 'binary',
    'metric': {'binary_logloss','auc'},
    'num_leaves': 128,#96,128,192,256,512
    'max_depth': 12,
    'num_threads':4,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.9,
    'bagging_freq': 5,
    'lambda_l1':60,
    'lambda_l2':30,
    'is_unbalance':True,
    'verbosity':-1,
    'bagging_seed':1294
}

model=lgb.train(params,train,num_boost_round=446)
pred=lgb.predict(Xl_df_test)

orders_df=pd.read_csv("orders.csv")
ramu=orders_df[orders_df["eval_set"]=="test"]
def mylist(df):
    ramu=df["product_id"]
    if ramu==[]: return 'None'
    return ' '.join(str(i) for i in ramu )
temp_df_test["in_cart"]=pred
temp_df_test.head()

test_set=temp_df_test[temp_df_test["in_cart"]>0.68]
test_set_list=test_set.groupby("order_id").agg({"product_id":(lambda x:list(x))})
test_set_list.head()
test_set_list=test_set_list.reset_index()
test_set_list['products_list']=test_set_list.apply(mylist,axis=1)
test_set_list.head()
sittam=test_set_list.merge(ramu,on="order_id",how="outer")
sittam=sittam.fillna('None')
sittam=sittam[["order_id","products_list"]]

sittam.to_csv("final_submission_lgb_0807_v6.csv")

NameError: name 'lgb' is not defined